In [ ]:
from torchvision import models
import torch
import pickle as pk # to save list as file
import gc # garbage collector, can help you free memory
!pip install import_ipynb
import import_ipynb # tool allow you to import from .ipynb files. Usualy import only from .py files.




In [ ]:
from create_dataset import DicomDataset # my dataset class
from create_dataset import train_metadata_df # import dataset csv

In [ ]:
# Creating instance of DicomDataset
from torchvision.transforms import ToTensor, ToPILImage


training_data = DicomDataset(
    merged_df=train_metadata_df,
    image_dir="train",
#    train=True,
#    download=True,
    transform=ToTensor()
)

In [ ]:
%whos # checking memory usage

In [ ]:

alexnet = models.alexnet(pretrained=True)
alexnet.eval()

# torch can use CUDA, this line check if CUDA is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


# from torchvision import transforms

# transform = transforms.Compose([            #[1]
#  transforms.Resize(256),                    #[2]
#  transforms.CenterCrop(224),                #[3]
#  transforms.ToTensor()
# ])

#img = Image.open("lot.png")
#img_t = transform(ToPILImage()((training_data[1])[0].to('cpu')))
#img_t = ((training_data[1])[0].to('cpu'))
#batch_t = torch.unsqueeze(img_t, 0)
#out = alexnet(batch_t)
#out = alexnet(train_dataloader)

ds_res= []
alexnet.to(device)
for image,lable in training_data:
    im_gpu = torch.unsqueeze(image,0).to(device)
    with torch.no_grad():
        im_res = alexnet(im_gpu)
    im_gpu.to('cpu')
    del im_gpu  # release memory, delete variable from memory
    print(len(ds_res))
    ds_res.append(im_res.to('cpu'))
    del im_res
    torch.cuda.empty_cache()
    gc.collect()


filename = "alexnet"
#save as binary file
pk.dump(ds_res, open(filename+".fea", 'wb'))
del alexnet
del ds_res
%whos

In [ ]:
model_dict = {"resnet50":models.resnet50(pretrained=True), 
"densenet161":models.densenet161(pretrained=True), 
"wide_resnet50_2":models.wide_resnet50_2(pretrained=True),
"vgg13":models.vgg13(pretrained=True)
}

for model_name in model_dict.keys():
    torch.cuda.empty_cache()
    model = model_dict[model_name]
    model.eval()
    model.to(device)
    
    ds_res= []
    print("start ",model_name)
    for image,lable in training_data:
        with torch.no_grad():
            im_res = model(torch.unsqueeze(image,0).to(device))
            
        ds_res.append(im_res.to('cpu'))
        del im_res
        torch.cuda.empty_cache()
        gc.collect()
        
    #save as binary file
    pk.dump(ds_res, open(model_name+".fea", 'wb'))
    del ds_res
    
